In [2]:
import tensorflow as tf
import os

2022-02-17 15:52:24.939249: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [11]:
import onnx

# Load the ONNX model
model = onnx.load("cnn.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a Human readable representation of the graph
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %actual_input[FLOAT, 1x1x28x28]\n) initializers (\n  %conv1.0.weight[FLOAT, 16x1x5x5]\n  %conv1.0.bias[FLOAT, 16]\n  %conv2.0.weight[FLOAT, 32x16x5x5]\n  %conv2.0.bias[FLOAT, 32]\n  %conv3.0.weight[FLOAT, 64x32x5x5]\n  %conv3.0.bias[FLOAT, 64]\n  %out.weight[FLOAT, 2x3136]\n  %out.bias[FLOAT, 2]\n  %26[INT64, 2]\n) {\n  %input = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%actual_input, %conv1.0.weight, %conv1.0.bias)\n  %10 = Relu(%input)\n  %input.4 = MaxPool[kernel_shape = [2, 2], pads = [0, 0, 0, 0], strides = [2, 2]](%10)\n  %input.8 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%input.4, %conv2.0.weight, %conv2.0.bias)\n  %13 = Relu(%input.8)\n  %input.12 = MaxPool[kernel_shape = [2, 2], pads = [0, 0, 0, 0], strides = [2, 2]](%13)\n  %input.16 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%in

In [12]:
tf_model_path = os.getcwd() + "/cnn-tf/"
tflite_model_path = tf_model_path + "saved_model.tflite"

In [5]:
with tf.device("/gpu:0"):
    model = tf.saved_model.load(tf_model_path)
    model.trainable = False

    input_tensor = tf.random.uniform([batch_size, channels, height, width])
    out = model(**{'input': input_tensor})

2022-02-17 15:52:27.869234: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-17 15:52:27.869851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-17 15:52:27.910385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 15:52:27.910567: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2022-02-17 15:52:27.910594: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-17 15:52:27.913078: I tensorflow/stream_executor/platform/def

NameError: name 'batch_size' is not defined

In [13]:
with tf.device("/gpu:0"):
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
    tflite_model = converter.convert()

    # Save the model
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)

2022-02-17 16:59:11.692327: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-02-17 16:59:11.692343: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2022-02-17 16:59:11.692346: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:325] Ignored change_concat_input_ranges.
2022-02-17 16:59:11.692444: I tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/cnn-tf/
2022-02-17 16:59:11.692845: I tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2022-02-17 16:59:11.692854: I tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: /home/hwixley/Documents/4th-Year/Honours-Project/localhost-data-preprocessing/cnn-tf/
2022-02-17 16:59:11.692877: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1261] Device interconnect Strea

In [ ]:
TF_PATH = "./deep-cnn-tf/saved_model.pb" # where the forzen graph is stored
TFLITE_PATH = "./deep-cnn-tf/saved_model.tflite"
# protopuf needs your virtual environment to be explictly exported in the path
os.environ["PATH"] = "/opt/miniconda3/envs/convert/bin:/opt/miniconda3/bin:/usr/local/sbin:...."

# make a converter object from the saved tensorflow file
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(TF_PATH,  # TensorFlow freezegraph .pb model file
                                                      input_arrays=['input'], # name of input arrays as defined in torch.onnx.export function before.
                                                      output_arrays=['output'] # name of output arrays defined in torch.onnx.export function before.
                                                      )

# tell converter which type of optimization techniques to use
# to view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]

converter.experimental_new_converter = True

# I had to explicitly state the ops
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]

tf_lite_model = converter.convert()
# Save the model.
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)

2022-02-17 15:43:31.820528: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-17 15:43:31.821035: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-17 15:43:31.863211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 15:43:31.863715: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2022-02-17 15:43:31.863773: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-17 15:43:31.896387: I tensorflow/stream_executor/platform/def